In [1]:
import spglib as spg
import ase
from ase.visualize import view

import ase.spacegroup as aspg
from ase.visualize import view
from ase.io import read,write
import numpy as np
import pandas as pd
from collections import Iterable
import cPickle as pck
import spglib as spg
import sys, os
from make_input.utils import rotation_matrix,isCellSkewed,unskewCell,get_symprec,get_relative_angle,isInSP
from make_input.qe_input import makeQEInput_new,makeCard,makeNamelist,SG2ibrav,SG2wyckoff,makeQEInput_sg,makeQEInput_ibrav0,makeQEInput
from make_input.custom_frame import frame2qe_format,get_ibrav2_frame,get_standard_frame,get_std_frame
from make_input.qe_run import run_qe_local
from make_input.raw_info import SG2BravaisLattice,missClassificationCorrection,dont_print_wyck,Ang2au
from read_output.parser import extract_floats,finishedProperly,fn2info,fn2num,get_filenames,get_patterns,get_energy_per_atom


In [2]:
import sys
sys.path.insert(0,'/local/git/glosim2/')
from libmatch.soap import get_Soaps
from libmatch.utils import get_soapSize,get_spkit,get_spkitMax,ase2qp,qp2ase
from libmatch.chemical_kernel import deltaKernel,PartialKernels
from GlobalSimilarity import get_environmentalKernels,get_globalKernel

# import info and input structures

In [3]:
fileNames = {}
infoPath = './info/'
structurePath = './structures/'
# fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
# fileNames['crystals'] = structurePath + 'input_crystals_sg1-230.pck'  
# fileNames['crystals'] = structurePath + 'input_crystals_sg1-230-17-10-17.pck'
# fileNames['crystals'] = structurePath + 'input_crystals_sg1-230-18-10-17.pck'
fileNames['crystals'] = structurePath + 'structures_101117.pck'
#fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
#fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
#fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [4]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
# with open(fileNames['wyck'],'rb') as f:
#     WyckTable = pck.load(f)
# SGTable = pd.read_pickle(fileNames['general info'])
# ElemTable = pd.read_pickle(fileNames['elements info'])

In [5]:
Ryd2eV = 13.605698066
Ang2au = 1.8897259885789

In [6]:
def compare(frame1,frame2):
    centerweight  = 1.
    gaussian_width = 0.1
    cutoff = 3.5
    cutoff_transition_width = 0.5
    nmax = 8
    lmax = 6
    nocenters = []
    is_fast_average = False

    soap_params = {
              'centerweight': centerweight, 
              'gaussian_width': gaussian_width,'cutoff': cutoff, 
              'cutoff_transition_width': cutoff_transition_width,
              'nmax': nmax, 'lmax': lmax, 'is_fast_average':is_fast_average,
              'chem_channels': True ,'nocenters': nocenters,'dispbar':True,
                   }
    ff = []
    for frame in [frame1,frame2]:
        ff.append(ase2qp(frame))
     
    envk = get_environmentalKernels(ff,nthreads=1, nprocess=1, nchunks=1,**soap_params)
    gkern = get_globalKernel(envk, kernel_type='average', zeta=4, gamma=1.0, eps=1e-06, 
                             nthreads=8, normalize_global_kernel=True)
    #print envk[(0,1)]
    return gkern[0,1]

# examples

In [10]:
from make_input.qe_input import frame2change,ibrav0,NOPROBLEM,dont_print_wyck

In [11]:
from make_input.raw_info import bravaisLattice2ibrav,SG2BravaisLattice

In [8]:
ssg = range(1,230+1)
sg2ibrav = {}
ibrav2sg = {ibrav:[] for ibrav in bravaisLattice2ibrav.values()}
for sg in ssg:
    bl = SG2BravaisLattice[sg]
    ibrav = bravaisLattice2ibrav[bl]
    ibrav2sg[ibrav].append(sg)
    sg2ibrav[sg] = ibrav

# test new input

In [12]:
from make_input.qe_input import makeQEInput_new

In [17]:
view(crystal)

In [36]:
from  tabulate import tabulate

In [135]:
data = [[1.,4.,5.],[4.,-6.],[3.,654.,7.,3.]]
for row in data:
    row_format ="{:>15.7f}" * (len(row) )
    print row_format.format( *row)

      1.0000000      4.0000000      5.0000000
      4.0000000     -6.0000000
      3.0000000    654.0000000      7.0000000      3.0000000


In [115]:
row_format

'{:.1f>7}{:.1f>7}{:.1f>7}'

In [75]:
aaa= np.random.rand(4,3)
aaa[0,2] = None
bbb = np.array(['Si','Ge','Al','Si'])
cc = {'a':aaa,'b':bbb}
orde = ['b','a']
ll = np.column_stack([cc[k] for k in orde])

print tabulate(ll, tablefmt="plain",floatfmt=".6f")

Si  0.582233  0.445679  nan
Ge  0.324148  0.714010    0.251025
Al  0.295494  0.238121    0.218413
Si  0.949544  0.150102    0.534996


In [33]:
crystal = crystals[67]
sites_z = [14]
inp_str = makeQEInput_new(crystal,sites_z,symprec=1e-5,
                    rhocutoff = None,wfccutoff = None,
                calculation_type='"scf"',smearing=1e-2,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                scf_conv_thr=1e-6,print_forces=True,print_stress=False,
                restart=False,collect_wf=False,force_ibrav0=False,
                kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],
                ppPath='"./pseudo/SSSP_acc_PBE/"')
print inp_str

&CONTROL 
  calculation = "scf" 
  outdir = "./out/" 
  wfcdir = "./wf_out/" 
  prefix = "qe" 
  pseudo_dir = "./pseudo/SSSP_acc_PBE/" 
  disk_io = "low" 
  restart_mode = "from_scratch" 
  verbosity = "high" 
  wf_collect = .false. 
  tprnfor = .true. 
  tstress = .false. 
  nstep = 150 
  etot_conv_thr = 0.00060000 
  forc_conv_thr = 0.00100000 
/ 
&SYSTEM 
  ecutrho = 560 
  ecutwfc = 70 
  ibrav = 4 
  nat = 1 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  degauss = 0.010000 
  space_group = 175 
  uniqueb = .FALSE. 
  A = 6.07285557 
  B = 6.07285556998 
  C = 9.079 
  cosAB = 0.00000 
  cosAC = 0.00000 
  cosBC = -0.50000 
/ 
&ELECTRONS 
  conv_thr = 0.00000100 
/ 
&CELL 
  press = 0.00000 
  press_conv_thr = 0.50000 
  cell_factor = 2.00000 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF   
ATOMIC_POSITIONS  crystal_sg 
  Si  6k    0.2843066   0.8572185 
K_POINTS  automatic 
      2    2    2    0    0    0 



## diamond silicon

In [10]:
fn = './structures/Si_mp-149_conventional_standard.cif'
cc = read(fn)

In [11]:
print spg.get_spacegroup(cc)

Fd-3m (227)


In [32]:
dd = get_standard_frame(cc,227)
view(dd)
print dd.get_cell_lengths_and_angles()
print cc.get_cell_lengths_and_angles()

[  3.86697462   3.86697462   3.86697462  60.          60.          60.        ]
[  5.46872796   5.46872796   5.46872796  90.          90.          90.        ]


In [12]:
data = spg.get_symmetry_dataset(cc,symprec=1e-5)
print data.keys()
print data['wyckoffs']

['std_positions', 'equivalent_atoms', 'std_types', 'translations', 'rotations', 'number', 'choice', 'transformation_matrix', 'std_mapping_to_primitive', 'origin_shift', 'hall_number', 'mapping_to_primitive', 'pointgroup', 'international', 'wyckoffs', 'hall', 'std_lattice']
['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']


In [18]:
zatom = 14
calculation_type = '"vc-relax"'

sg = 227

kpt = [2,2,2]
Nkpt = 1000
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 60*8,60
smearing = 1e-6
etot_conv_thr = 1e-4
forc_conv_thr = 1e-4
nstep = 150
scf_conv_thr = 1e-6
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_diamond/'
dirName = dataPath 

In [22]:
fn = './structures/Si_mp-149_conventional_standard.cif'
cc = read(fn)



input_str = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                calculation_type=calculation_type,smearing=smearing,
                pressure=0,press_conv_thr=0.5,cell_factor=3,
                etot_conv_thr=etot_conv_thr,forc_conv_thr=forc_conv_thr,nstep=nstep,
                scf_conv_thr=scf_conv_thr,print_forces=True,
                kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                ppPath=ppPath)
#print input_str

fnout = dirName+'qe.in'
with open(fnout,'w') as f:
    f.write(input_str)

#print 'sending the calc'
#exitstatus = run_qe_local(input_str,dirName,verbose=False,
#             path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

#print exitstatus

&CONTROL 
  calculation = "vc-relax" 
  outdir = "./out/" 
  wfcdir = "./wf_out/" 
  prefix = "qe" 
  pseudo_dir = "/local/git/run_qe/pseudo/SSSP_acc_PBE/" 
  disk_io = "low" 
  restart_mode = "from_scratch" 
  verbosity = "high" 
  wf_collect = .false. 
  tprnfor = .true. 
  tstress = .true. 
  nstep = 150 
  etot_conv_thr = 0.00020000 
  forc_conv_thr = 0.00010000 
/ 
&SYSTEM 
  ecutrho = 480 
  ecutwfc = 60 
  ibrav = 2 
  nat = 1 
  ntyp = 1 
  occupations = "smearing" 
  smearing = "cold" 
  degauss = 0.000001 
  space_group = 227 
  uniqueb = .FALSE. 
  rhombohedral = .TRUE. 
  A = 5.46872796 
  B = 5.46872796 
  C = 5.46872796 
  cosAB = 0.00000 
  cosAC = 0.00000 
  cosBC = 0.00000 
/ 
&ELECTRONS 
  conv_thr = 0.00000100 
/ 
&IONS 
  ion_dynamics = "bfgs" 
/ 
&CELL 
  press = 0.00000 
  press_conv_thr = 0.50000 
  cell_factor = 3.00000 
/ 
ATOMIC_SPECIES   
  Si  28.0855  Si.pbe-n-rrkjus_psl.1.0.0.UPF 
ATOMIC_POSITIONS  crystal_sg 
  Si     0.   0.   0.5 
K_POINTS  automatic 
 

In [37]:
fn = dirName+'qe.out'
ccs = read(fn,index='-1')
view(ccs)


In [38]:
print ccs.get_cell_lengths_and_angles()

[  7.31022221   7.31022221   7.31022221  60.          60.          60.        ]


In [30]:
1.889725989*5.46872796

10.334397352782952

In [34]:
0.529177249*4.477

2.369126543773

In [40]:
from ase.units import create_units
units = create_units('2006')
print units['Bohr']

0.529177208354


In [50]:
def string2index(string):
    if ':' not in string:
        return int(string)
    i = []
    for s in string.split(':'):
        if s == '':
            i.append(None)
        else:
            i.append(int(s))
    i += (3 - len(i)) * [None]
    return slice(*i)
def parse_filename(filename, index=None):
    if not isinstance(filename, basestring) or '@' not in filename:
        return filename, index
    newindex = None
    if ('.json@' in filename or
        '.db@' in filename or
        filename.startswith('pg://')):
        newfilename, newindex = filename.rsplit('@', 1)
    else:
        newfilename, newindex = filename.rsplit('@', 1)
        try:
            newindex = string2index(newindex)
        except ValueError:
            return filename, index
    return newfilename, newindex


In [62]:
from read_output.qe_reader import read_qe
fn = dirName+'qe.out'
ccs = read_qe(fn)
for cc in ccs:
    print cc.get_volume()
    print cc.get_positions()

40.8882783694
[[-1.36718184  1.36718184  1.36718184]
 [-2.73436368  2.73436368  2.73436368]]
40.9336177967
[[-1.36768699  1.36768699  1.36768699]
 [-2.73537398  2.73537398  2.73537398]]
40.9336177967
[[-1.36768699  1.36768699  1.36768699]
 [-2.73537398  2.73537398  2.73537398]]


In [65]:
view(ccs)

In [64]:
sys.executable

'/home/musil/miniconda/envs/glosim/bin/python'

## single sg

In [22]:
sg,it = 210,0
cc = crystals[sg][it]
dirName = './test_run/check_input_errors_ibrav3/sg_{}-f_{}'.format(sg,it)


In [23]:
print cc.get_scaled_positions()[0]

[ 0.30321532  0.3032168   0.58934866]


In [24]:
input_str = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                scf_conv_thr=1e-1,print_forces=True,print_stress=False,
                restart=False,
                 kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],ppPath='"../pseudo/"')
# print inpt

In [ ]:
exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

## several sg

In [19]:
zatom = 14
calculation_type = '"scf"'
sgs = [202]

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"./pseudo/SSSP_acc_PBE/"'

dataPath = './test_run/check_input_errors_ibrav3/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [20]:
for (sg,it),dirName in dirNames.iteritems():
    print dirName
    crystal = crystals[sg][it]
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

./test_run/check_input_errors_ibrav3/sg_202-f_0
sending the calc
(0, 'No error')


# ibrav 3 test

In [38]:
sg,it = 220,0
cc = crystals[sg][it]
dirName = './test_run/check_input_errors_ibrav3/sg_{}-f_{}'.format(sg,it)

In [29]:
print cc.get_scaled_positions()[0]

[ 0.62501026  0.76843214  0.48157344]


In [33]:
fns = get_filenames('./test_run/check_input_errors_ibrav3/', fn_pattern='qe.out', dir_pattern=None)
fns

['./test_run/check_input_errors_ibrav3/sg_220-f_0-0/qe.out',
 './test_run/check_input_errors_ibrav3/sg_202-f_0-1/qe.out']

In [45]:
fn = './test_run/check_input_errors_ibrav3/sg_220-f_0-0/qe.out'
oo = read(fn)
view(oo)

In [37]:
print oo.get_cell()

[[ 4.22005118  4.22005118  4.22005118]
 [-4.22005118  4.22005118  4.22005118]
 [-4.22005118 -4.22005118  4.22005118]]


In [44]:
dd,_,_ = get_ibrav2_frame(cc,220)
view(dd)

# ibrav 9 test

## preliminary

In [54]:
dataPath = './test_run/check_input_errors_ibrav9/'

In [55]:
sg,it = 40,0
cc = crystals[sg][it]
dirName = dataPath+'sg_{}-f_{}/'.format(sg,it)

In [60]:
view(cc)

In [56]:
print cc.get_cell_lengths_and_angles()
print cc.get_scaled_positions()[0]

[  8.10960373   3.90162918   2.25168312  90.          90.          90.        ]
[ 0.61485391  0.83332859  0.49      ]


In [57]:
fns = get_filenames(dataPath, fn_pattern='qe.out', dir_pattern=None)
fns

['./test_run/check_input_errors_ibrav9/sg_40-f_0-0/qe.out']

In [59]:
fn = './test_run/check_input_errors_ibrav9/sg_40-f_0-0/qe.out'
oo = read(fn)
view(oo)

In [37]:
print oo.get_cell()

[[ 4.22005118  4.22005118  4.22005118]
 [-4.22005118  4.22005118  4.22005118]
 [-4.22005118 -4.22005118  4.22005118]]


In [44]:
dd,_,_ = get_ibrav2_frame(cc,220)
view(dd)

## full

In [383]:
zatom = 14
calculation_type = '"scf"'
sgs = [20, 21, 35, 36, 37, 38, 39, 40, 41, 63, 64, 65, 66, 67, 68]
sgs = [35, 65, 68]

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav9/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [385]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    data = spg.get_symmetry_dataset(crystal,symprec=symprec)
    
    if len(np.unique(data['equivalent_atoms'])) > 1:
        print '########'
        print sg,it
        print '########'
        continue
    elif sg == 35 and data['wyckoffs'][0] not in ['p','q']:
        continue
    
    print dirName,'  ',data['wyckoffs'][0]
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_3    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_2    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_1    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_0    c
sending the calc
(0, 'No error')
########
8 5
########
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_4    d
sending the calc
(0, 'No error')


In [134]:
missClassificationCorrection[35]

65

In [431]:
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_8*')
fns

['/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_4-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_0-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_1-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_2-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_3-0/qe.out']

In [432]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [433]:
failed = {}
for fn,(sg,f) in zip(fns,ppp):
    if not finishedProperly(fn):
        with open(fn,'r') as fout:
            failed[(sg,f,fn)] = fout.read()
print len(failed)

0


In [440]:
sg,it = 8,0
cc = crystals[sg][it]
dd = get_standard_frame(cc,sg,primitive=False)
view(dd)
print dd.get_cell()
print dd.get_scaled_positions()
fn = fnd[(sg,it)]
print fn
rr = read(fn)
ss = get_standard_frame(rr,sg,primitive=False)
view(ss)
print ss.get_cell()
print ss.get_scaled_positions()

[[ 3.91330814  0.          0.        ]
 [ 0.          2.26301179  0.        ]
 [-1.30763156  0.          1.84589388]]
[[ -2.83704473e-17   0.00000000e+00   5.00000000e-01]
 [  5.00000000e-01   5.00000000e-01   5.00000000e-01]]
/local/git/run_qe/test_run/check_input_errors_ibrav9/sg_8-f_0-0/qe.out
[[ 3.91330635  0.          0.        ]
 [ 0.          2.26300958  0.        ]
 [-1.30763147  0.          1.84589338]]
[[ 0.5  0.5  0. ]
 [ 0.   0.   0. ]]


In [169]:
dd.get_scaled_positions()

array([[ 0.5       ,  0.74998098,  0.        ],
       [ 0.5       ,  0.25001902,  0.        ],
       [ 0.24997246,  0.        ,  0.5       ],
       [ 0.75002754,  0.        ,  0.5       ],
       [ 0.        ,  0.24998098,  0.        ],
       [ 0.        ,  0.75001902,  0.        ],
       [ 0.74997246,  0.5       ,  0.5       ],
       [ 0.25002754,  0.5       ,  0.5       ]])

In [173]:
spg.get_symmetry_dataset?

In [177]:
spg.get_spacegroup(cc,symprec=1e-4)

u'Cmmm (65)'

In [249]:
data = spg.get_symmetry_dataset(cc,symprec=1e-4)
print data['number'],data['wyckoffs'],data['equivalent_atoms']
data.keys()
# data

65 ['i', 'i', 'h', 'h', 'h', 'h', 'i', 'i'] [0 0 2 2 2 2 0 0]


['std_positions',
 'equivalent_atoms',
 'std_types',
 'translations',
 'rotations',
 'number',
 'choice',
 'transformation_matrix',
 'origin_shift',
 'hall_number',
 'pointgroup',
 'international',
 'wyckoffs',
 'hall',
 'std_lattice']

In [263]:
sg,it = 65,1
cc = crystals[sg][it]
print cc.get_scaled_positions()[0]
print cc.get_cell_lengths_and_angles()
print isInSP(cc,sg)
symprec = get_symprec(cc,sg)
data = spg.get_symmetry_dataset(cc,symprec=symprec)
print data['number'],data['wyckoffs'],data['equivalent_atoms']
print data['std_positions']
view(cc)

[ 0.12500016  0.06249997  0.87500515]
[   6.78590329    4.52128294    4.52096173   90.00000062   90.00000016
  109.45930452]
True
65 ['k', 'n', 'q', 'q', 'p', 'p', 'l', 'n', 'n', 'k', 'p', 'p', 'q', 'q', 'n', 'l'] [0 1 2 2 4 4 6 1 1 0 4 4 2 2 1 6]
[[ 0.          0.          0.74999485]
 [ 0.5         0.75000016  0.75000329]
 [ 0.25000119  0.624999    0.5       ]
 [ 0.24999881  0.875001    0.5       ]
 [ 0.75000091  0.62500133  0.        ]
 [ 0.74999909  0.87499867  0.        ]
 [ 0.5         0.          0.25000104]
 [ 0.          0.74999984  0.24999671]
 [ 0.5         0.75000016  0.24999671]
 [ 0.          0.          0.25000515]
 [ 0.25000091  0.87499867  0.        ]
 [ 0.24999909  0.62500133  0.        ]
 [ 0.75000119  0.875001    0.5       ]
 [ 0.74999881  0.624999    0.5       ]
 [ 0.          0.74999984  0.75000329]
 [ 0.5         0.          0.74999896]
 [ 0.5         0.5         0.74999485]
 [ 0.          0.25000016  0.75000329]
 [ 0.75000119  0.124999    0.5       ]
 [ 0.749998

In [91]:
spg.get_spacegroup(rr,symprec=)

u'Cmcm (63)'

In [193]:
ss = spg.get_symmetry_from_database?

In [213]:
ss = aspg.crystal(symbols=['Si'],basis=[0.3548618,0.3548618,0.1],cellpar=[8 ,   8,  3 , 90.  ,        90.    ,      90],spacegroup=35)
view(ss)

(0, 0, 0)+  (1/2, 1/2, 0)+  
(1) x, y, z	(2) -x, -y, z	(3) x, -y, z	(4) -x, y, z

In [207]:
sg,it = 35,1
cc = crystals[sg][it]
print cc.get_scaled_positions()

[[ 0.6451382   0.64752901  0.216     ]
 [ 0.3548618   0.35247099  0.216     ]
 [ 0.6451382   0.35247099  0.216     ]
 [ 0.3548618   0.64752901  0.216     ]
 [ 0.1451382   0.14752901  0.216     ]
 [ 0.8548618   0.85247099  0.216     ]
 [ 0.1451382   0.85247099  0.216     ]
 [ 0.8548618   0.14752901  0.216     ]]


In [82]:
print dd.get_cell_lengths_and_angles()
print dd.get_scaled_positions()[0]

[  3.12444082   7.72951641   3.1713511   90.          90.          90.        ]
[ 0.24999444  0.60755706  0.25000888]


# check wyckoff pos

In [5]:
def get_primitive(frame,sg):
    symprec = get_symprec(frame, sg)
    if symprec is None:
        print 'Not possible'
        return None
    (lattice, positions, numbers) = spg.standardize_cell(
                            frame, to_primitive=False,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
    primitive_atoms = ase.Atoms(cell=lattice, scaled_positions=positions, numbers=numbers)
    return primitive_atoms

In [6]:
highestMultiplicitySite = WyckTable[50].loc[0,:]
wl = highestMultiplicitySite['Wyckoff letter']
print wl

m


In [10]:
def match_with_ase(crystal,sg,tol=1e-5):
    Ncc = crystal.get_number_of_atoms()
    ee = aspg.crystal(symbols=['Si'],basis=crystal.get_scaled_positions()[0],cell=crystal.get_cell(),spacegroup=sg)
    Nee = ee.get_number_of_atoms()
    if Ncc == Nee:
        aa = np.allclose(ee.get_scaled_positions(),crystal.get_scaled_positions(),atol=tol)
        bb = np.allclose(ee.get_cell(),crystal.get_cell(),atol=tol)
        return aa and bb
    else:
        return False

In [55]:
# sg,it = not_unique[0]
sg,it = 224,0
print sg,it
crystal = crystals[sg][it]
ee = aspg.crystal(symbols=['Si'],basis=crystal.get_scaled_positions()[0],cell=crystal.get_cell(),spacegroup=sg)
print np.allclose(ee.get_scaled_positions(),crystal.get_scaled_positions(),atol=1e-5)
view(crystal)
view(ee)

224 0
True


In [53]:
sg,it = 228,2
print sg,it
crystal = crystals[sg][it]
print spg.get_spacegroup(crystal,symprec=1e-5)
print match_with_ase(crystal,sg,tol=1e-5)

228 2
Fd-3c (228)
True


In [51]:
try:
    missClassificationCorrection[sg]
except:
    print 'not in the list'
# symprec = get_symprec(crystal,sg)
spg.get_spacegroup(crystal,symprec=1e-2)

not in the list


u'Im-3m (229)'

In [33]:
symprec = get_symprec(crystal,sg)
data = spg.get_symmetry_dataset(crystal,symprec=symprec)
print data['number'],data['equivalent_atoms'],data['wyckoffs']

10 [0 0 0 0] ['o', 'o', 'o', 'o']


In [57]:
wrong_wl

[(12, 4),
 (51, 0),
 (51, 1),
 (51, 2),
 (51, 4),
 (65, 3),
 (65, 4),
 (72, 8),
 (83, 2),
 (123, 1),
 (123, 3),
 (123, 4),
 (127, 4),
 (174, 2),
 (175, 0),
 (191, 0),
 (215, 2),
 (217, 2),
 (224, 1),
 (227, 1)]

In [36]:
not_unique = []
miss = []
wrong_wl = []
good = []
tot = 0
for sg,cl in crystals.iteritems():
    highestMultiplicitySite = WyckTable[sg].loc[0,:]
    wl = highestMultiplicitySite['Wyckoff letter']
    for it,crystal in enumerate(cl):
        tot += 1
        symprec = get_symprec(crystal,sg)
        if symprec is None:
            print sg,it
            continue
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
#         print data['number'],data['wyckoffs'],data['equivalent_atoms']
        if sg in missClassificationCorrection.keys():
            miss.append((sg,it))
        elif len(np.unique(data['equivalent_atoms'])) > 1:
            not_unique.append((sg,it))
        elif data['wyckoffs'][0] != wl:
            wrong_wl.append((sg,it))
        else:
            good.append((sg,it))
Nu,Nm,Nw,Ng = len(not_unique),len(miss),len(wrong_wl),len(good)
print Nu,Nm,Nw,Ng,tot

7 0
7 2
7 3
8 4
11 1
11 3
12 1
13 0
13 1
13 2
13 6
14 0
15 1
15 2
15 4
15 5
15 6
15 7
15 8
15 9
15 14
16 0
16 1
16 2
16 3
16 4
17 0
17 1
17 2
17 3
17 4
18 0
18 1
18 2
18 3
18 4
19 0
19 1
19 2
19 4
20 0
20 1
20 2
20 3
20 4
20 5
20 7
20 8
20 9
20 10
20 11
20 12
20 13
21 0
21 1
21 2
21 3
21 4
21 5
21 6
21 8
21 9
22 0
22 3
23 0
23 1
23 2
23 3
23 4
24 0
24 1
24 2
24 3
24 4
26 0
26 3
27 1
27 3
27 4
28 4
29 2
29 3
30 0
30 1
30 3
31 0
31 4
31 6
33 1
33 3
34 1
34 4
35 4
36 1
36 6
37 0
37 1
37 2
37 7
38 0
38 4
39 0
39 2
39 3
39 4
40 0
40 1
40 4
40 6
41 1
41 3
41 7
42 1
43 3
43 4
43 5
44 0
45 0
45 1
45 2
45 3
48 1
48 2
48 3
48 4
48 5
48 7
50 0
50 1
50 2
50 3
50 4
52 1
52 2
52 5
52 6
52 7
52 8
52 9
52 10
52 11
52 12
53 0
53 1
53 4
54 0
54 2
54 3
54 4
55 0
55 1
55 2
56 2
56 4
56 5
57 0
57 1
57 2
57 4
57 5
57 6
57 7
57 8
58 0
58 1
58 3
58 4
59 0
59 2
59 3
59 4
60 0
60 1
60 3
60 4
61 0
61 1
61 2
61 4
62 1
62 2
62 3
62 4
62 5
62 6
62 8
62 9
63 1
63 2
64 0
64 2
64 3
64 4
65 0
65 1
65 2
66 0
66 1
66 3
6

In [273]:
print Nu,Nm,Nw,Ng,tot

68 283 66 797 1214


In [309]:
len(missClassificationCorrection)

53

### test pretty inspection

In [62]:
from ipywidgets import Layout, Button, Box,HTML
from bs4 import BeautifulSoup
sg = 4
# url of space group info at International Tables for Crystallography Volume A: Space-group symmetry
baseUrl = 'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o'
url = baseUrl + '{:03.0f}'.format(sg)+'/'

# get the xml from the web page
soup = BeautifulSoup(requests.get(url).text, "lxml")

# get list of table of class genpos (-> general position part) and specpos (-> special positions)
genposList = soup.find_all("table", class_="genpos")
specposList = soup.find_all("table", class_="specpos")
genposcoords = soup.find_all("table", class_="genposcoords")
aa = genposList[0]
#     aa += specposList[1]
aa.append(specposList[0])

items = [
    HTML(aa.prettify())
 ]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='50%')
box = Box(children=items, layout=box_layout)
box



NameError: name 'BeautifulSoup' is not defined

In [64]:
def get_sg_table(sg):
    import requests
    from bs4 import BeautifulSoup
    from IPython.display import HTML

    # url of space group info at International Tables for Crystallography Volume A: Space-group symmetry
    baseUrl = 'http://it.iucr.org/Ac/ch2o3v0001/sgtable2o3o'
    url = baseUrl + '{:03.0f}'.format(sg)+'/'

    # get the xml from the web page
    soup = BeautifulSoup(requests.get(url).text, "lxml")

    # get list of table of class genpos (-> general position part) and specpos (-> special positions)
    genposList = soup.find_all("table", class_="genpos")
    specposList = soup.find_all("table", class_="specpos")
    aa = genposList[0]
    if len(specposList) > 0:
        aa.append(specposList[0])
    return aa

def print_sg_table(sg1,sg2):
    from ipywidgets import Layout, Button, Box,HTML
    from bs4 import BeautifulSoup
    t1 = get_sg_table(sg1)
    t2 = get_sg_table(sg2)
    items = [HTML(t1.prettify()),HTML(t2.prettify())]

    box_layout = Layout(display='flex',
                        flex_flow='row',
                        align_items='stretch',
                        border='solid',
                        width='100%')
    box = Box(children=items, layout=box_layout)
    return box
print_sg_table(10,12)

# ibrav 9 test new

In [91]:
missClassificationCorrection[64]

KeyError: 64

In [123]:
zatom = 14
calculation_type = '"scf"'
sgs = [20, 21, 35, 36, 37, 38, 39, 40, 41, 63, 64, 65, 66, 67, 68]
# sgs = [35, 65, 68]
# sgs = [38, 39, 40, 41, 63, 64, 65, 66, 67, 68]
# sgs = [64]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [124]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_40-f_3    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_64-f_0    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_36-f_6    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_41-f_2    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_38-f_1    n
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_39-f_4    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_35-f_0    q
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_41-f_8    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_40-f_0    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_ne

(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_36-f_1    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_41-f_7    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_38-f_4    n
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_37-f_4    l
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_67-f_3    o
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_39-f_1    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_63-f_4    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_20-f_2    c
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_68-f_4    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_

In [125]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
fns

['/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_40-f_0-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_21-f_3-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_36-f_2-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_66-f_0-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_36-f_7-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_40-f_8-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_64-f_3-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_37-f_5-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_68-f_3-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_40-f_7-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_21-f_2-0/qe.out',
 '/local/git/run_qe/test_run/check_input_errors_ibrav9_new/sg_64-f_1-0/qe.out',
 '/local/git/run_qe/test_run/check_input

In [128]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [130]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
4 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=178), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=15931), HTML(value=u'')))


[0, 1, 40, 0] 0.999999999976
[2, 3, 21, 3] 0.999999999907
[4, 5, 36, 2] 0.999999999962
[6, 7, 66, 0] 1.0
[8, 9, 36, 7] 0.999999999943
[10, 11, 40, 8] 0.999999999988
[12, 13, 64, 3] 0.999999999941
[14, 15, 37, 5] 1.0
[16, 17, 68, 3] 1.0
[18, 19, 40, 7] 0.999999999961
[20, 21, 21, 2] 0.999999999985
[22, 23, 64, 1] 0.999999999945
[24, 25, 41, 2] 0.999999999999
[26, 27, 68, 5] 0.999999999999
[28, 29, 67, 2] 0.999999999971
[30, 31, 35, 4] 0.999999999999
[32, 33, 64, 2] 0.999999999997
[34, 35, 35, 1] 0.999999999997
[36, 37, 41, 3] 0.999999999997
[38, 39, 38, 0] 0.999999999997
[40, 41, 35, 2] 0.999999999999
[42, 43, 40, 1] 0.999999999938
[44, 45, 39, 1] 0.999999999997
[46, 47, 41, 5] 0.999999999997
[48, 49, 21, 1] 1.0
[50, 51, 39, 2] 1.0
[52, 53, 41, 7] 0.999999999994
[54, 55, 20, 5] 0.999999999999
[56, 57, 40, 10] 0.999999999984
[58, 59, 63, 1] 1.0
[60, 61, 21, 4] 0.999999999999
[62, 63, 63, 4] 0.999999999992
[64, 65, 35, 3] 0.999999999997
[66, 67, 67, 1] 0.999999999971
[68, 69, 39, 0] 0.99

In [111]:
for it,fn in enumerate(fns):
    
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    
    if f in [3,]:
        cc  = read(fn)
        view(cc)
        dd = crystals[sg][f]
        view(dd)
        raw_input('press enter')

press enter


In [173]:
spg.get_symmetry_dataset?

In [177]:
spg.get_spacegroup(cc,symprec=1e-4)

u'Cmmm (65)'

In [249]:
data = spg.get_symmetry_dataset(cc,symprec=1e-4)
print data['number'],data['wyckoffs'],data['equivalent_atoms']
data.keys()
# data

65 ['i', 'i', 'h', 'h', 'h', 'h', 'i', 'i'] [0 0 2 2 2 2 0 0]


['std_positions',
 'equivalent_atoms',
 'std_types',
 'translations',
 'rotations',
 'number',
 'choice',
 'transformation_matrix',
 'origin_shift',
 'hall_number',
 'pointgroup',
 'international',
 'wyckoffs',
 'hall',
 'std_lattice']

In [263]:
sg,it = 65,1
cc = crystals[sg][it]
print cc.get_scaled_positions()[0]
print cc.get_cell_lengths_and_angles()
print isInSP(cc,sg)
symprec = get_symprec(cc,sg)
data = spg.get_symmetry_dataset(cc,symprec=symprec)
print data['number'],data['wyckoffs'],data['equivalent_atoms']
print data['std_positions']
view(cc)

[ 0.12500016  0.06249997  0.87500515]
[   6.78590329    4.52128294    4.52096173   90.00000062   90.00000016
  109.45930452]
True
65 ['k', 'n', 'q', 'q', 'p', 'p', 'l', 'n', 'n', 'k', 'p', 'p', 'q', 'q', 'n', 'l'] [0 1 2 2 4 4 6 1 1 0 4 4 2 2 1 6]
[[ 0.          0.          0.74999485]
 [ 0.5         0.75000016  0.75000329]
 [ 0.25000119  0.624999    0.5       ]
 [ 0.24999881  0.875001    0.5       ]
 [ 0.75000091  0.62500133  0.        ]
 [ 0.74999909  0.87499867  0.        ]
 [ 0.5         0.          0.25000104]
 [ 0.          0.74999984  0.24999671]
 [ 0.5         0.75000016  0.24999671]
 [ 0.          0.          0.25000515]
 [ 0.25000091  0.87499867  0.        ]
 [ 0.24999909  0.62500133  0.        ]
 [ 0.75000119  0.875001    0.5       ]
 [ 0.74999881  0.624999    0.5       ]
 [ 0.          0.74999984  0.75000329]
 [ 0.5         0.          0.74999896]
 [ 0.5         0.5         0.74999485]
 [ 0.          0.25000016  0.75000329]
 [ 0.75000119  0.124999    0.5       ]
 [ 0.749998

In [91]:
spg.get_spacegroup(rr,symprec=)

u'Cmcm (63)'

In [193]:
ss = spg.get_symmetry_from_database?

In [213]:
ss = aspg.crystal(symbols=['Si'],basis=[0.3548618,0.3548618,0.1],cellpar=[8 ,   8,  3 , 90.  ,        90.    ,      90],spacegroup=35)
view(ss)

(0, 0, 0)+  (1/2, 1/2, 0)+  
(1) x, y, z	(2) -x, -y, z	(3) x, -y, z	(4) -x, y, z

In [207]:
sg,it = 35,1
cc = crystals[sg][it]
print cc.get_scaled_positions()

[[ 0.6451382   0.64752901  0.216     ]
 [ 0.3548618   0.35247099  0.216     ]
 [ 0.6451382   0.35247099  0.216     ]
 [ 0.3548618   0.64752901  0.216     ]
 [ 0.1451382   0.14752901  0.216     ]
 [ 0.8548618   0.85247099  0.216     ]
 [ 0.1451382   0.85247099  0.216     ]
 [ 0.8548618   0.14752901  0.216     ]]


In [82]:
print dd.get_cell_lengths_and_angles()
print dd.get_scaled_positions()[0]

[  3.12444082   7.72951641   3.1713511   90.          90.          90.        ]
[ 0.24999444  0.60755706  0.25000888]


In [273]:
print Nu,Nm,Nw,Ng,tot

68 283 66 797 1214


In [309]:
len(missClassificationCorrection)

53

# ibrav 8 test

In [143]:
from make_input.qe_input import tricky_sg,ibrav0
print tricky_sg,ibrav0

[] []


In [91]:
missClassificationCorrection[64]

KeyError: 64

In [144]:
zatom = 14
calculation_type = '"scf"'
sgs = [16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 47, 48, 49, 50, 51,
                   52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
# sgs = [35, 65, 68]
# sgs = [38, 39, 40, 41, 63, 64, 65, 66, 67, 68]
# sgs = [64]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav8/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [145]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_59-f_0    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_31-f_6    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_62-f_2    d
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_19-f_4    a
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_18-f_4    c
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_48-f_1    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_25-f_1    u
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_27-f_2    q
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_51-f_1    l
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_32-f_2    h
sending the calc
(0, 'N

(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_55-f_0    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_60-f_0    d
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_18-f_1    c
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_25-f_4    x
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_26-f_3    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_50-f_4    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_56-f_4    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_28-f_2    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_52-f_7    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav8/sg_57-f_1    e
sending

In [147]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [148]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [149]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=300), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=45150), HTML(value=u'')))


[0, 1, 17, 4] 0.999999999997
[2, 3, 33, 0] 0.999999999996
[4, 5, 27, 1] 0.999999999998
[6, 7, 60, 0] 0.999999999999
[8, 9, 61, 0] 0.999999999998
[10, 11, 50, 2] 0.999999999999
[12, 13, 61, 4] 1.0
[14, 15, 53, 4] 0.999999999995
[16, 17, 61, 3] 1.0
[18, 19, 33, 3] 1.0
[20, 21, 62, 3] 0.999999999998
[22, 23, 19, 4] 1.0
[24, 25, 51, 4] 0.999999999992
[26, 27, 33, 4] 0.999999999999
[28, 29, 50, 3] 0.999999999999
[30, 31, 50, 0] 0.999999999986
[32, 33, 56, 4] 0.999999999995
[34, 35, 19, 2] 0.999999999996
[36, 37, 55, 2] 0.999999999999
[38, 39, 56, 1] 0.999999999998
[40, 41, 28, 1] 0.999999999997
[42, 43, 25, 4] 1.0
[44, 45, 55, 3] 1.0
[46, 47, 52, 1] 0.999999999997
[48, 49, 18, 0] 0.999999999999
[50, 51, 51, 1] 1.0
[52, 53, 54, 0] 0.999999999999
[54, 55, 32, 3] 1.0
[56, 57, 57, 0] 0.999999999999
[58, 59, 31, 3] 0.999999999999
[60, 61, 17, 2] 1.0
[62, 63, 30, 3] 1.0
[64, 65, 54, 4] 1.0
[66, 67, 29, 3] 0.999999999996
[68, 69, 53, 0] 0.999999999999
[70, 71, 51, 0] 0.999999999999
[72, 73, 57, 2

# ibrav -12 test

In [8]:
from make_input.qe_input import tricky_sg,ibrav0
print tricky_sg,ibrav0

[] [1, 2, 5, 8, 9, 12, 15]


In [91]:
missClassificationCorrection[64]

KeyError: 64

In [150]:
zatom = 14
calculation_type = '"scf"'
sgs = [3, 4, 6, 7, 10, 11, 13, 14]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav12/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [151]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_7-f_3    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_3-f_0    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_14-f_4    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_11-f_2    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_13-f_5    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_6-f_2    l
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_14-f_0    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_13-f_2    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_10-f_3    o
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav12/sg_7-f_2    e
sending the calc


In [152]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [153]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [154]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=90), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=4095), HTML(value=u'')))


[0, 1, 3, 0] 0.999999999878
[2, 3, 3, 4] 0.999999999328
[4, 5, 4, 0] 0.999999999985
[6, 7, 7, 2] 0.999999999892
[8, 9, 4, 1] 1.0
[10, 11, 10, 0] 0.999999999625
[12, 13, 10, 4] 0.999999999777
[14, 15, 7, 1] 0.999999999999
[16, 17, 6, 2] 0.999999674079
[18, 19, 13, 6] 0.999999999996
[20, 21, 11, 2] 0.999999999938
[22, 23, 10, 1] 0.999999998567
[24, 25, 14, 0] 0.99999999992
[26, 27, 14, 1] 0.999999999966
[28, 29, 6, 4] 0.999999961068
[30, 31, 11, 1] 0.99999999953
[32, 33, 13, 5] 0.999999999978
[34, 35, 13, 0] 0.999999999997
[36, 37, 6, 1] 0.999999999836
[38, 39, 3, 5] 0.999999999992
[40, 41, 7, 4] 0.999999999991
[42, 43, 13, 4] 0.999999999997
[44, 45, 11, 4] 0.999999999861
[46, 47, 3, 1] 0.999999999942
[48, 49, 13, 3] 0.999999999776
[50, 51, 3, 3] 0.999999999977
[52, 53, 11, 3] 0.999999999798
[54, 55, 4, 5] 0.999999999745
[56, 57, 3, 2] 0.999999999946
[58, 59, 10, 3] 0.999999999922
[60, 61, 4, 2] 0.999999995831
[62, 63, 3, 6] 0.999999999786
[64, 65, 6, 0] 0.999999999996
[66, 67, 11, 0] 0

# ibrav 5 test

In [143]:
from make_input.qe_input import tricky_sg,ibrav0
print tricky_sg,ibrav0

[] []


In [91]:
missClassificationCorrection[64]

KeyError: 64

In [167]:
zatom = 14
calculation_type = '"scf"'
sgs = [3, 4, 6, 7, 10, 11, 13, 14]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav5/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [168]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_7-f_3    f
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_3-f_0    m
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_14-f_4    e
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_11-f_2    f
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_13-f_5    g
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_6-f_2    l
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_14-f_0    e
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_13-f_2    g
ASDFGASDFGA
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_10-f_3    o
ASDFGASDFGA
sending the calc
(0

In [169]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [170]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [171]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=90), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=4095), HTML(value=u'')))


[0, 1, 3, 0] 0.999999999878
[2, 3, 3, 4] 0.999999999328
[4, 5, 4, 0] 0.999999999985
[6, 7, 7, 2] 0.999999999892
[8, 9, 4, 1] 1.0
[10, 11, 10, 0] 0.999999999625
[12, 13, 10, 4] 0.999999999777
[14, 15, 7, 1] 0.999999999999
[16, 17, 6, 2] 0.999999674079
[18, 19, 13, 6] 0.999999999996
[20, 21, 11, 2] 0.999999999938
[22, 23, 10, 1] 0.999999998567
[24, 25, 14, 0] 0.99999999992
[26, 27, 14, 1] 0.999999999966
[28, 29, 6, 4] 0.999999961068
[30, 31, 11, 1] 0.99999999953
[32, 33, 13, 5] 0.999999999978
[34, 35, 13, 0] 0.999999999997
[36, 37, 6, 1] 0.999999999836
[38, 39, 3, 5] 0.999999999992
[40, 41, 7, 4] 0.999999999991
[42, 43, 13, 4] 0.999999999997
[44, 45, 11, 4] 0.999999999861
[46, 47, 3, 1] 0.999999999942
[48, 49, 13, 3] 0.999999999776
[50, 51, 3, 3] 0.999999999977
[52, 53, 11, 3] 0.999999999798
[54, 55, 4, 5] 0.999999999745
[56, 57, 3, 2] 0.999999999946
[58, 59, 10, 3] 0.999999999922
[60, 61, 4, 2] 0.999999995831
[62, 63, 3, 6] 0.999999999786
[64, 65, 6, 0] 0.999999999996
[66, 67, 11, 0] 0

# ibrav 3 test

In [143]:
from make_input.qe_input import tricky_sg,ibrav0
print tricky_sg,ibrav0

[] []


In [91]:
missClassificationCorrection[64]

KeyError: 64

In [172]:
zatom = 14
calculation_type = '"scf"'
sgs = [197, 199, 204, 206, 211, 214, 217, 220, 229, 230]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav3_new/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [173]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_230-f_3    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_206-f_1    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_211-f_2    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_229-f_0    l
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_214-f_0    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_199-f_0    c
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_204-f_0    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_230-f_0    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav3_new/sg_220-f_0    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_

In [174]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [175]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [176]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=50), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=1275), HTML(value=u'')))


[0, 1, 220, 0] 1.0
[2, 3, 211, 3] 1.0
[4, 5, 204, 1] 0.999999999999
[6, 7, 204, 0] 1.0
[8, 9, 230, 3] 1.0
[10, 11, 214, 1] 1.0
[12, 13, 206, 0] 1.0
[14, 15, 197, 0] 1.0
[16, 17, 197, 1] 1.0
[18, 19, 211, 1] 1.0
[20, 21, 230, 0] 1.0
[22, 23, 229, 1] 1.0
[24, 25, 217, 0] 0.999999999999
[26, 27, 214, 2] 1.0
[28, 29, 229, 0] 1.0
[30, 31, 220, 2] 1.0
[32, 33, 199, 0] 1.0
[34, 35, 217, 1] 1.0
[36, 37, 211, 2] 1.0
[38, 39, 211, 0] 1.0
[40, 41, 206, 1] 1.0
[42, 43, 230, 1] 1.0
[44, 45, 230, 2] 1.0
[46, 47, 214, 0] 1.0
[48, 49, 220, 1] 1.0


# ibrav 2 test

In [179]:
zatom = 14
calculation_type = '"scf"'
sgs = [196, 202, 203, 209, 210, 216, 219, 225, 226, 227, 228]
sgs = [ 225, 226]
kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav2/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [180]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav2/sg_226-f_0    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav2/sg_225-f_0    l
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav2/sg_226-f_1    j
sending the calc
(0, 'No error')


In [181]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [182]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [183]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=38), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=741), HTML(value=u'')))


[0, 1, 216, 0] 1.0
[2, 3, 225, 0] 1.0
[4, 5, 203, 1] 1.0
[6, 7, 228, 0] 1.0
[8, 9, 210, 2] 1.0
[10, 11, 202, 0] 1.0
[12, 13, 228, 2] 1.0
[14, 15, 196, 0] 1.0
[16, 17, 219, 0] 1.0
[18, 19, 228, 4] 0.999999999999
[20, 21, 210, 1] 1.0
[22, 23, 226, 1] 1.0
[24, 25, 227, 0] 1.0
[26, 27, 226, 0] 1.0
[28, 29, 210, 0] 1.0
[30, 31, 203, 0] 1.0
[32, 33, 228, 1] 1.0
[34, 35, 209, 0] 1.0
[36, 37, 228, 3] 1.0


# ibrav 14 test
set to ibrav = 0

In [191]:
zatom = 14
calculation_type = '"scf"'
sgs = [1,2]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav14/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [192]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_1-f_2    a
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_2-f_7    i
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_2-f_5    i
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_1-f_3    d
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_1-f_4    e
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_2-f_4    i
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_2-f_0    i
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_2-f_6    i
NONONONO
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav14/sg_2-f_3    i
NONONONO
sending the calc
(0, 'No error')
/local/git

In [195]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*-1')
# fns

In [196]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [197]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=26), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=351), HTML(value=u'')))


[0, 1, 2, 4] 1.0
[2, 3, 2, 3] 1.0
[4, 5, 1, 4] 0.999999999998
[6, 7, 2, 6] 1.0
[8, 9, 2, 0] 0.999999999998
[10, 11, 1, 2] 1.0
[12, 13, 1, 0] 0.99999999999
[14, 15, 2, 1] 0.999999999996
[16, 17, 2, 2] 0.999999999998
[18, 19, 2, 7] 0.999999999999
[20, 21, 1, 1] 0.999999999989
[22, 23, 2, 5] 0.999999999997
[24, 25, 1, 3] 1.0


# ibrav 13 test
ibrav = 0
https://www.mail-archive.com/pw_forum@pwscf.org/msg29900.html
9 not implemented and look at sg 5 f 0 for some trial

In [58]:
zatom = 14
calculation_type = '"scf"'
sgs = [5, 8, 9, 12, 15]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav13_2/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [209]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_12-f_1    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_9-f_1    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_8-f_0    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_1    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_8    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_9-f_4    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_5-f_1    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_5    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_12-f_2    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_9-f_0    

In [210]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*-11')
#fns

In [211]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [212]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    
    gk = compare(cc,dd)
    print gk,fn


0.999999999615 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_7-11/qe.out
0.999999997874 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_12-f_1-11/qe.out
0.999999999904 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_4-11/qe.out
0.999999999952 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_8-f_4-11/qe.out
0.999999999736 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_5-f_3-11/qe.out
0.999999999751 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_11-11/qe.out
0.999999999971 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_6-11/qe.out
0.999999998955 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_5-f_4-11/qe.out
0.999999997403 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_8-f_3-11/qe.out
0.999999999747 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_15-f_9-11/qe.out
0.999999986844 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_9-f_2-11/qe.out
0.9

In [194]:
sg,f = 8,0
fn = '/local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_{}-f_{}-8/qe.out'.format(sg,f)
cc  = read(fn)
    
dd = crystals[sg][f]

In [195]:
gk = compare(cc,dd)
print gk

0.999999962354


In [164]:
print dd.get_cell()
print dd.get_cell_lengths_and_angles()

[[  4.48952179e+00   2.22107842e-07  -2.37599450e-01]
 [  5.19841279e-07   4.39853288e+00  -1.82665367e-07]
 [ -7.75428562e+00  -5.01581064e-07   2.35852004e+00]]
[   4.49580465    4.39853288    8.10503315   90.00001072  166.1119401
   89.99999028]


In [173]:
aa = dd.get_cell_lengths_and_angles()[:3]*Ang2au
print aa
print aa / aa[0]
bb = np.cos(dd.get_cell_lengths_and_angles()[3:]*np.pi/180)
print bb

[ 4.24534023  8.3120219   7.35707655]
[ 1.          1.95791655  1.7329769 ]
[ -1.93516745e-07  -4.76884612e-04   4.08225659e-09]


In [153]:
ee = get_standard_frame(dd,sg)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()

[[ 2.24697439 -2.19926644  0.        ]
 [ 2.24697439  2.19926644  0.        ]
 [-1.12212219  0.          1.94622002]]
[   3.14414802    3.14414802    2.24653746  110.91372052  110.91372052
   88.77048522]


In [177]:
print cc.get_cell()
print cc.get_cell_lengths_and_angles()

[[  1.12326858e+00  -2.19926553e+00   0.00000000e+00]
 [  1.12326858e+00   2.19926553e+00   0.00000000e+00]
 [ -1.85563970e-03   0.00000000e+00   3.89319724e+00]]
[   2.46951437    2.46951437    3.89319769   90.01242172   90.01242172
  125.88891348]


In [196]:
view(cc)
view(unskewCell(dd))

# ibrav 11 test

In [282]:
zatom = 14
calculation_type = '"scf"'
sgs = [23, 24, 44, 45, 46, 71, 72, 73, 74]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav11/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [283]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_23-f_4    k
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_45-f_1    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_72-f_2    k
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_44-f_0    l
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_73-f_3    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_24-f_4    d
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_23-f_3    k
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_44-f_4    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_74-f_1    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav11/sg_46-f_3    h
sending the c

In [284]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [285]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [18]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    
    gk = compare(cc,dd)
    print gk,fn


0.703068269406 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_12-f_1-1/qe.out
0.850385128672 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_12-f_1-0/qe.out
0.543074821606 /local/git/run_qe/test_run/check_input_errors_ibrav13_2/sg_12-f_1-2/qe.out


In [20]:

ee = get_standard_frame(dd,sg)
print ee.get_cell()
print ee.get_scaled_positions()
print ee.get_cell_lengths_and_angles()

[[ 2.24757171 -2.20045643  0.        ]
 [ 2.24757171  2.20045643  0.        ]
 [-2.24631253  0.          3.89130715]]
[[ 0.25001955  0.75004275  0.75005237]
 [ 0.24995725  0.74998045  0.24994763]
 [ 0.74998045  0.24995725  0.24994763]
 [ 0.75004275  0.25001955  0.75005237]]
[   3.1454073     3.1454073     4.49312712  110.93069348  110.93069348
   88.78624607]


In [21]:
print cc.get_cell()
print cc.get_scaled_positions()
print cc.get_cell_lengths_and_angles()

[[ 1.57270355  0.         -2.24656299]
 [-1.12366523  2.93785112  0.        ]
 [ 1.57270355  0.          2.24656299]]
[[ 0.75002137  0.74998785  0.25004055]
 [ 0.25004055  0.74998785  0.75002137]
 [ 0.7499593   0.25001172  0.24997848]
 [ 0.24997848  0.25001172  0.7499593 ]]
[   2.74234238    3.1454082     2.74234238  101.82207557  110.01221251
  101.82207557]


In [22]:
view(cc)
view(ee)

# ibrav 10 test

In [287]:
zatom = 14
calculation_type = '"scf"'
sgs = [22, 42, 43, 69, 70]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav10/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [288]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_70-f_4    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_69-f_4    p
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_43-f_3    b
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_42-f_0    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_70-f_5    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_69-f_3    p
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_70-f_3    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_22-f_2    k
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_43-f_2    b
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav10/sg_70-f_6    h
sending the c

In [289]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [290]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [291]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=56), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=1596), HTML(value=u'')))


[0, 1, 70, 4] 0.99999999998
[2, 3, 70, 5] 0.999999999747
[4, 5, 43, 1] 0.999999999976
[6, 7, 43, 3] 0.999999999784
[8, 9, 69, 0] 0.999999999999
[10, 11, 70, 3] 1.0
[12, 13, 42, 2] 0.999999999994
[14, 15, 69, 3] 0.999999999996
[16, 17, 42, 0] 0.999999999836
[18, 19, 43, 0] 0.999999999683
[20, 21, 42, 3] 0.999999999991
[22, 23, 22, 1] 1.0
[24, 25, 43, 5] 0.999999999863
[26, 27, 70, 1] 1.0
[28, 29, 22, 2] 1.0
[30, 31, 42, 1] 0.999999999975
[32, 33, 43, 2] 0.999999999991
[34, 35, 69, 4] 1.0
[36, 37, 70, 0] 0.999999999763
[38, 39, 22, 3] 1.0
[40, 41, 22, 0] 0.999999999998
[42, 43, 43, 4] 0.999999999756
[44, 45, 70, 2] 0.999999999972
[46, 47, 69, 2] 0.999999999997
[48, 49, 42, 4] 0.999999999915
[50, 51, 70, 6] 0.999999999986
[52, 53, 69, 1] 0.999999999996
[54, 55, 22, 4] 0.999999999999


# ibrav 7 test

In [292]:
zatom = 14
calculation_type = '"scf"'
sgs = [79, 80, 82, 87, 88, 97, 98, 107, 108, 109, 110, 119, 120, 121, 122,
                   139, 140, 141, 142]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav7/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [293]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_108-f_2    k
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_109-f_3    c
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_97-f_5    k
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_121-f_0    j
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_98-f_2    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_140-f_2    m
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_82-f_3    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_79-f_2    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_141-f_3    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_79-f_4    h
sending the calc
(

(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_87-f_1    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_120-f_1    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_97-f_1    k
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_142-f_3    g
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_107-f_0    l
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_88-f_3    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_110-f_2    b
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_87-f_4    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_122-f_1    e
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7/sg_79-f_1    h
se

In [294]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [295]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [296]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=200), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=20100), HTML(value=u'')))


[0, 1, 79, 3] 0.999999999998
[2, 3, 120, 3] 0.999999999933
[4, 5, 108, 3] 0.999999999986
[6, 7, 82, 0] 0.999999999996
[8, 9, 88, 0] 0.999999999967
[10, 11, 121, 4] 0.99999999998
[12, 13, 98, 3] 0.999999999994
[14, 15, 119, 2] 0.999999999999
[16, 17, 88, 1] 0.999999999997
[18, 19, 97, 0] 0.999999999999
[20, 21, 119, 3] 0.999999999995
[22, 23, 119, 0] 0.999999999997
[24, 25, 82, 6] 0.999999999991
[26, 27, 80, 4] 0.999999999962
[28, 29, 139, 2] 0.999999999986
[30, 31, 79, 4] 0.999999999999
[32, 33, 122, 3] 0.999999999935
[34, 35, 98, 0] 0.999999999994
[36, 37, 142, 0] 0.999999999998
[38, 39, 108, 4] 0.9999999999
[40, 41, 122, 0] 0.999999999998
[42, 43, 121, 0] 0.999999999996
[44, 45, 119, 4] 0.999999999826
[46, 47, 109, 4] 0.999999999893
[48, 49, 108, 0] 0.999999999981
[50, 51, 97, 1] 0.999999999958
[52, 53, 88, 2] 0.999999999997
[54, 55, 87, 1] 0.999999999999
[56, 57, 139, 3] 0.999999999999
[58, 59, 139, 0] 0.999999999967
[60, 61, 87, 3] 0.999999999998
[62, 63, 120, 0] 0.999999999898
[6

# ibrav 5 test

In [67]:
zatom = 14
calculation_type = '"scf"'
sgs = [146, 148, 155, 160, 161, 166, 167]

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav5/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [23]:
sg,it = 160,0
crystal = crystals[sg][it]
fn = '/home/musil/source/qe-6.1/PW/tools/structures/sg_{}-f_{}.cif'.format(sg,it)
write(fn,crystal)

In [68]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_160-f_1    c
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_146-f_0    b
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_167-f_0    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_148-f_3    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_166-f_1    i
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_146-f_4    b
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_167-f_4    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_148-f_7    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_155-f_2    f
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav5/sg_167-f_8    f
sending the c

In [69]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [70]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [71]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=90), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=4095), HTML(value=u'')))


[0, 1, 155, 2] 0.999999999908
[2, 3, 146, 4] 0.999999983135
[4, 5, 148, 5] 0.999999994229
[6, 7, 148, 6] 0.999999999955
[8, 9, 161, 0] 0.999999966873
[10, 11, 166, 1] 0.999999990557
[12, 13, 160, 0] 0.999999999972
[14, 15, 148, 3] 0.999999999658
[16, 17, 155, 0] 0.999999999678
[18, 19, 148, 1] 0.999999998351
[20, 21, 167, 3] 0.999999937521
[22, 23, 148, 7] 0.999999999812
[24, 25, 146, 2] 0.999999999998
[26, 27, 161, 4] 0.99999999997
[28, 29, 167, 7] 0.999999990479
[30, 31, 155, 4] 0.999999999988
[32, 33, 167, 2] 0.999999998634
[34, 35, 148, 0] 0.999999999912
[36, 37, 166, 4] 0.999999976032
[38, 39, 167, 6] 0.999999999446
[40, 41, 160, 3] 0.999999999968
[42, 43, 155, 3] 0.999999984381
[44, 45, 160, 4] 0.999999999888
[46, 47, 161, 2] 0.999999999986
[48, 49, 155, 1] 0.999999999966
[50, 51, 161, 3] 0.99999999827
[52, 53, 167, 5] 0.999999999928
[54, 55, 167, 9] 0.999999999998
[56, 57, 166, 3] 0.999999998702
[58, 59, 167, 8] 0.999999983262
[60, 61, 146, 3] 0.999999999998
[62, 63, 146, 0] 0.

In [61]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    if sg == 146 and f==0:
        cc  = read(fn)
    
        dd = crystals[sg][f]
        print cc.get_cell()
        print cc.get_cell_lengths_and_angles()
        print cc.get_scaled_positions()
        
        print dd.get_cell()
        print dd.get_cell_lengths_and_angles()
        print dd.get_scaled_positions()
        
        view(cc)
        view(dd)

[[ 1.94611209 -1.12358936  2.2005419 ]
 [ 0.          2.24717872  2.2005419 ]
 [-1.94611209 -1.12358936  2.2005419 ]]
[  3.1451852    3.1451863    3.1451852   76.45119927  76.45115344
  76.45119927]
[[ 0.84627831  0.51302207  0.17970004]
 [ 0.17970018  0.84627805  0.5130222 ]
 [ 0.51302194  0.1797003   0.84627818]]
[[  3.89222573e+00  -3.80321834e-16   5.48433444e-17]
 [ -1.94611286e+00   3.37076636e+00  -3.87945386e-17]
 [  1.43356282e-16  -3.08447024e-17   6.60162771e+00]]
[   3.89222573    3.89222573    6.60162771   90.           90.          120.        ]
[[  3.33365822e-01   2.19299536e-05   4.87000000e-01]
 [  9.99978070e-01   3.33343892e-01   4.87000000e-01]
 [  6.66656108e-01   6.66634178e-01   4.87000000e-01]
 [  3.24883071e-05   3.33355263e-01   8.20333333e-01]
 [  6.66644737e-01   6.66677225e-01   8.20333333e-01]
 [  3.33322775e-01   9.99967512e-01   8.20333333e-01]
 [  6.66699155e-01   6.66688597e-01   1.53666667e-01]
 [  3.33311403e-01   1.05583535e-05   1.53666667e-01]
 [

In [62]:
gk = compare([cc,dd])
print gk[0,1]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=2), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=3), HTML(value=u'')))


0.999999999999


In [57]:
np.cos(np.pi/180.*76.45116809)

0.23427400808223917

In [38]:
ee = get_standard_frame(dd,146)
print ee.get_cell()
print ee.get_cell_lengths_and_angles()
print ee.get_scaled_positions() 
view(ee)

[[ 1.94611286  1.12358879  2.20054257]
 [-1.94611286  1.12358879  2.20054257]
 [ 0.         -2.24717757  2.20054257]]
[  3.14518595   3.14518595   3.14518595  76.45116809  76.45116809
  76.45116809]
[[ 0.15365611  0.48697807  0.82036582]
 [ 0.82036582  0.15365611  0.48697807]
 [ 0.48697807  0.82036582  0.15365611]]


# ibrav 7 test relax

In [310]:
zatom = 14
calculation_type = '"vc-relax"'
sgs = [79, 80, 82, 87, 88, 97, 98, 107, 108, 109, 110, 119, 120, 121, 122,
                   139, 140, 141, 142]
sgs = [79]

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-0
forc_conv_thr = 1e-0
nstep = 150
scf_conv_thr = 1e-0
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [311]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=etot_conv_thr,forc_conv_thr=forc_conv_thr,nstep=nstep,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_4    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_3    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_0    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_2    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_1    h
sending the calc
(0, 'No error')


In [294]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [295]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [296]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc,dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=200), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=20100), HTML(value=u'')))


[0, 1, 79, 3] 0.999999999998
[2, 3, 120, 3] 0.999999999933
[4, 5, 108, 3] 0.999999999986
[6, 7, 82, 0] 0.999999999996
[8, 9, 88, 0] 0.999999999967
[10, 11, 121, 4] 0.99999999998
[12, 13, 98, 3] 0.999999999994
[14, 15, 119, 2] 0.999999999999
[16, 17, 88, 1] 0.999999999997
[18, 19, 97, 0] 0.999999999999
[20, 21, 119, 3] 0.999999999995
[22, 23, 119, 0] 0.999999999997
[24, 25, 82, 6] 0.999999999991
[26, 27, 80, 4] 0.999999999962
[28, 29, 139, 2] 0.999999999986
[30, 31, 79, 4] 0.999999999999
[32, 33, 122, 3] 0.999999999935
[34, 35, 98, 0] 0.999999999994
[36, 37, 142, 0] 0.999999999998
[38, 39, 108, 4] 0.9999999999
[40, 41, 122, 0] 0.999999999998
[42, 43, 121, 0] 0.999999999996
[44, 45, 119, 4] 0.999999999826
[46, 47, 109, 4] 0.999999999893
[48, 49, 108, 0] 0.999999999981
[50, 51, 97, 1] 0.999999999958
[52, 53, 88, 2] 0.999999999997
[54, 55, 87, 1] 0.999999999999
[56, 57, 139, 3] 0.999999999999
[58, 59, 139, 0] 0.999999999967
[60, 61, 87, 3] 0.999999999998
[62, 63, 120, 0] 0.999999999898
[6

# ibrav 7 test relax

In [313]:
zatom = 14
calculation_type = '"vc-relax"'
sgs = [79, 80, 82, 87, 88, 97, 98, 107, 108, 109, 110, 119, 120, 121, 122,
                   139, 140, 141, 142]
sgs = [79]

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-0
forc_conv_thr = 1e-0
nstep = 150
scf_conv_thr = 1e-0
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [ ]:
for (sg,it),dirName in dirNames.items():
    
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=etot_conv_thr,forc_conv_thr=forc_conv_thr,nstep=nstep,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_4    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_3    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_0    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_2    h
sending the calc
(0, 'No error')
/local/git/run_qe/test_run/check_input_errors_ibrav7_relax/sg_79-f_1    h
sending the calc


In [314]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [315]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [ ]:
def parse_structural_similarity(fns):
    idd = []
    fails =  []

    for it,fn in enumerate(fns):
        info = fn2info(fn)
        sg = info['sg']
        f = info['f']
        try:
            cc  = read(fn,index=':')

            dd = crystals[sg][f]
            #ee = get_standard_frame(dd,sg)
            sim = compare(cc[0],dd)

            idd.append([sg,f,sim])

        except:
            print sg,f,fn
            fails.append([sg,f,fn])
    return idd,fails

In [325]:

idd = []
fails =  []

for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    try:
        cc  = read(fn,index=':')

        dd = crystals[sg][f]
        #ee = get_standard_frame(dd,sg)
        sim = compare(cc[0],dd)
        
        idd.append([sg,f,sim])
        
    except:
        print sg,f,fn
        fails.append([sg,f,fn])

for it in idd:
    print it

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=12), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=78), HTML(value=u'')))


[0, 1, 79, 3] 0.999999999998
[2, 3, 79, 4] 0.999999999994
[4, 5, 79, 4] 0.999999999999
[6, 7, 79, 0] 0.999999999981
[8, 9, 79, 2] 0.999999999918
[10, 11, 79, 1] 0.999999999977


In [324]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    if f == 0:
        cc  = read(fn,index=':')
        
        dd = crystals[sg][f]
        view(cc[0])
        view(dd)

#  test all scf

In [7]:
sites_z = [14]
calculation_type = '"scf"'
sgs = range(1,230+1)

kpt = [1,1,1]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 10
forc_conv_thr = 10
nstep = 150
scf_conv_thr = 10
symprec = 1e-5
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_error_scf_4/'
dirNames = {(it):dataPath + 'idx_{}'.format(it)
            for it,crystal in enumerate(crystals)}

In [34]:
exitstatus = {}
for (it),dirName in dirNames.items():

    crystal = crystals[it]
    
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',it,data['number'],data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput_new(crystal,sites_z,symprec=symprec,
                    rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                calculation_type=calculation_type,smearing=smearing,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=etot_conv_thr,forc_conv_thr=forc_conv_thr,nstep=nstep,
                scf_conv_thr=scf_conv_thr,print_forces=False,print_stress=False,
                restart=False,collect_wf=False,force_ibrav0=False,
                kpt = kpt,Nkpt=None,kpt_offset = [0,0,0],
                ppPath='"./pseudo/SSSP_acc_PBE/"')

    print 'sending the calc'
    exitstatus[it] = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus[it][0]

/local/git/run_qe/test_run/check_input_error_scf_4/idx_67    67 175 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_68    68 2 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_69    69 12 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_70    70 227 f
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_71    71 123 m
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_72    72 194 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_73    73 221 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_74    74 229 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_75    75 160 b
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_76    76 181 j
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_77    77 192 m
sending the calc
0
/local/git/run_qe/test_r

0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_158    158 180 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_159    159 166 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_160    160 178 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_161    161 141 a
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_162    162 172 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_163    163 228 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_164    164 12 b
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_165    165 70 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_166    166 88 f
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_167    167 211 j
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_168    168 88 f
sending the calc
0
/

0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_249    249 227 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_250    250 214 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_251    251 230 e
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_252    252 69 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_253    253 224 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_254    254 229 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_255    255 227 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_256    256 69 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_257    257 130 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_258    258 142 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_259    259 194 c
sending the calc
0

0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_340    340 230 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_341    341 123 m
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_342    342 229 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_343    343 47 q
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_344    344 210 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_345    345 230 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_346    346 203 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_347    347 203 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_348    348 220 e
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_349    349 229 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_350    350 204 h
sending the calc


0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_430    430 167 f
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_431    431 230 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_432    432 229 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_433    433 176 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_434    434 167 e
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_435    435 69 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_436    436 142 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_437    437 225 f
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_438    438 227 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_439    439 12 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_440    440 167 e
sending the calc
0

0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_521    521 203 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_522    522 210 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_523    523 142 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_524    524 230 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_525    525 203 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_526    526 181 j
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_527    527 229 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_528    528 191 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_529    529 206 e
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_530    530 139 j
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_531    531 141 h
sending the calc

0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_612    612 166 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_613    613 179 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_614    614 226 j
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_615    615 212 b
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_616    616 142 f
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_617    617 222 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_618    618 225 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_619    619 136 f
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_620    620 226 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_621    621 209 j
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_622    622 210 h
sending the calc

0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_703    703 208 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_704    704 142 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_705    705 176 h
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_706    706 221 m
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_707    707 142 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_708    708 180 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_709    709 229 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_710    710 194 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_711    711 220 e
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_712    712 69 i
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_713    713 206 e
sending the calc


0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_794    794 127 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_795    795 228 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_796    796 203 g
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_797    797 140 k
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_798    798 179 c
sending the calc
0
/local/git/run_qe/test_run/check_input_error_scf_4/idx_799    799 70 g
sending the calc
0


In [16]:
for it,cc in enumerate(crystals):
    data = spg.get_symmetry_dataset(cc,symprec=1e-5)
    if data['number'] == 160 and data['wyckoffs'] == 'b':
        print it

In [17]:
from ase.spacegroup import crystal
crystal?

In [24]:
x,z = 0.1,0.6
gg = crystal(symbols=[14], basis=[x,-x,z], spacegroup=160, 
             cellpar=[7.20980787,7.20980787, 4.4830964 , 90.,90., 119.99999997])
view(gg)
hh = get_std_frame(gg)
print hh.get_scaled_positions()[0]
view(hh)

[ 0.4  0.7  0.7]


In [19]:
from tabulate import tabulate
from scipy.spatial.distance import cdist
cc = crystals[56]

# print tabulate(cc.get_scaled_positions())
# print tabulate(cdist(cc.get_scaled_positions(),cc.get_scaled_positions()))

In [21]:
cc.get_cell_lengths_and_angles()

array([   7.20980787,    7.20980787,    4.4830964 ,   90.        ,
         90.        ,  119.99999997])

In [10]:
data = spg.get_symmetry_dataset(cc,symprec=1e-5)
print data['number'],data['wyckoffs']

166 ['h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h', 'h']


In [11]:
[146, 148, 155, 160, 161, 166, 167]

[146, 148, 155, 160, 161, 166, 167]

In [12]:
ee = get_std_frame(cc)
view(ee)

In [13]:
print ee.get_scaled_positions()[0]
par = ee.get_cell_lengths_and_angles()
print par
print par[0]*Ang2au
print par[:3]/par[0]
print np.cos(par[3:]*np.pi/180.)

[ 0.75000015  0.75000015  0.25000135]
[   4.42269578    4.42269578    4.42269578  109.19291858  109.19291858
  109.19291858]
8.35768315904
[ 1.  1.  1.]
[-0.32874992 -0.32874992 -0.32874992]


In [163]:
view(cc)

In [14]:
fn = '/local/git/run_qe/test_run/check_input_error_scf_4/idx_56-7/qe.out'
dd = read(fn)

In [15]:
compare(cc,dd)

0.99999999999332989

In [20]:
view(dd)

In [162]:
to_look_at = [56,68]
print exitstatus[56][1]


     Program PWSCF v.6.1 (svn rev. 13369) starts on 10Nov2017 at 15:35:27 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Parallel version (MPI), running on     4 processors
     R & G space division:  proc/nbgrp/npool/nimage =       4
     Reading input from /local/git/run_qe/test_run/check_input_error_scf_4/idx_56-0/qe.in

     Current dimensions of program PWSCF are:
     Max number of different atomic species (ntypx) = 10
     Max number of k-points (npk) =  40000
     Max angular momentum in pseudopotentials (lmaxx) =  3

     Subspace diagonalization in iterative solution of the eigenvalue problem:
     a serial algorithm will be used


 %%%%%%%%%%%%%%%%%%%%%

In [152]:
dont_print_wyck

[(33, '4a'),
 (226, '8a'),
 (227, '8a'),
 (4, '2a'),
 (7, '2a'),
 (9, '4a'),
 (170, '6a'),
 (225, '4a'),
 (76, '4a'),
 (78, '4a'),
 (144, '3a'),
 (145, '3a'),
 (19, '4a'),
 (169, '6a'),
 (228, '16a'),
 (29, '4a')]

In [153]:
(227,'8a') in dont_print_wyck

True

In [363]:
fails_overlap_atm = [[160,0],[166,0]]

In [224]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0])) 166 160
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
fns[:5]

['/local/git/run_qe/test_run/check_input_error_scf_3/sg_215-f_0-0/qe.out',
 '/local/git/run_qe/test_run/check_input_error_scf_3/sg_79-f_3-0/qe.out',
 '/local/git/run_qe/test_run/check_input_error_scf_3/sg_74-f_1-0/qe.out',
 '/local/git/run_qe/test_run/check_input_error_scf_3/sg_150-f_2-0/qe.out',
 '/local/git/run_qe/test_run/check_input_error_scf_3/sg_3-f_0-0/qe.out']

In [225]:
params = {'sg':[],'f':[]}
ppp = []
fnd = {}

for fn in fns:
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    ppp.append([sg,f])
    fnd[(sg,f)] = fn

In [226]:
fff = []
idd = []
for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn)
    
    dd = crystals[sg][f]
    
    gk = compare(cc,dd)
    print gk,fn


0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_215-f_0-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_79-f_3-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_74-f_1-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_150-f_2-0/qe.out
0.999999999878 /local/git/run_qe/test_run/check_input_error_scf_3/sg_3-f_0-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_76-f_1-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_89-f_0-0/qe.out
0.999999999328 /local/git/run_qe/test_run/check_input_error_scf_3/sg_3-f_4-0/qe.out
0.999999999994 /local/git/run_qe/test_run/check_input_error_scf_3/sg_91-f_4-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_201-f_1-0/qe.out
0.999999999985 /local/git/run_qe/test_run/check_input_error_scf_3/sg_5-f_1-0/qe.out
0.999999999996 /local/git/run_qe/test_run/check_input_error_scf_3/sg_16-f_1-1/qe.out
0.999999998674 /local/gi

1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_203-f_1-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_62-f_3-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_221-f_0-0/qe.out
0.99999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_15-f_8-0/qe.out
0.999999999955 /local/git/run_qe/test_run/check_input_error_scf_3/sg_148-f_6-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_154-f_7-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_19-f_4-0/qe.out
0.999999999941 /local/git/run_qe/test_run/check_input_error_scf_3/sg_64-f_3-0/qe.out
0.999999999992 /local/git/run_qe/test_run/check_input_error_scf_3/sg_3-f_5-1/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_22-f_2-1/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_228-f_0-0/qe.out
0.999999999976 /local/git/run_qe/test_run/check_input_error_scf_3/sg_43-f_1-0/qe.out
1.0 /local/git/run_qe/test_run/c

1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_206-f_0-0/qe.out
0.999999999988 /local/git/run_qe/test_run/check_input_error_scf_3/sg_168-f_4-0/qe.out
0.999999999328 /local/git/run_qe/test_run/check_input_error_scf_3/sg_3-f_4-1/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_119-f_0-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_178-f_4-0/qe.out
0.999999999985 /local/git/run_qe/test_run/check_input_error_scf_3/sg_176-f_0-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_93-f_0-0/qe.out
0.999999999972 /local/git/run_qe/test_run/check_input_error_scf_3/sg_160-f_0-0/qe.out
0.999999999971 /local/git/run_qe/test_run/check_input_error_scf_3/sg_67-f_2-0/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_52-f_1-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_202-f_0-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_35-f_4-0/qe.out
0.99999

1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_218-f_1-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_182-f_3-0/qe.out
0.999999999996 /local/git/run_qe/test_run/check_input_error_scf_3/sg_13-f_6-0/qe.out
0.999999999988 /local/git/run_qe/test_run/check_input_error_scf_3/sg_118-f_1-0/qe.out
0.999999999986 /local/git/run_qe/test_run/check_input_error_scf_3/sg_149-f_3-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_15-f_0-1/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_218-f_0-0/qe.out
0.99999999859 /local/git/run_qe/test_run/check_input_error_scf_3/sg_9-f_0-0/qe.out
0.999999999915 /local/git/run_qe/test_run/check_input_error_scf_3/sg_29-f_2-1/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_85-f_4-0/qe.out
0.999999999985 /local/git/run_qe/test_run/check_input_error_scf_3/sg_73-f_7-0/qe.out
0.999999999987 /local/git/run_qe/test_run/check_input_error_scf_3/sg_105-f_2-0/qe.out
1.0 /local/git/run_q

0.999999999992 /local/git/run_qe/test_run/check_input_error_scf_3/sg_188-f_3-0/qe.out
0.999999999995 /local/git/run_qe/test_run/check_input_error_scf_3/sg_150-f_0-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_191-f_3-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_21-f_4-0/qe.out
0.999999999967 /local/git/run_qe/test_run/check_input_error_scf_3/sg_139-f_0-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_133-f_1-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_205-f_0-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_213-f_1-0/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_114-f_1-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_20-f_2-1/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_87-f_3-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_27-f_4-1/qe.out
0.999999999999 /local/git/r

...

0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_95-f_4-0/qe.out
0.999999999798 /local/git/run_qe/test_run/check_input_error_scf_3/sg_11-f_3-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_94-f_0-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_165-f_4-0/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_64-f_4-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_174-f_4-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_116-f_0-0/qe.out
0.999999999991 /local/git/run_qe/test_run/check_input_error_scf_3/sg_20-f_1-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_134-f_4-0/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_13-f_0-1/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_2-f_2-1/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_154-f_3-0/qe.ou

0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_63-f_2-0/qe.out
0.999999999996 /local/git/run_qe/test_run/check_input_error_scf_3/sg_35-f_0-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_120-f_1-0/qe.out
0.999999999644 /local/git/run_qe/test_run/check_input_error_scf_3/sg_11-f_0-0/qe.out
0.999999999947 /local/git/run_qe/test_run/check_input_error_scf_3/sg_98-f_2-0/qe.out
0.999999999988 /local/git/run_qe/test_run/check_input_error_scf_3/sg_156-f_4-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_2-f_7-1/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_14-f_2-0/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_193-f_4-0/qe.out
0.999999999992 /local/git/run_qe/test_run/check_input_error_scf_3/sg_14-f_4-0/qe.out
0.999999999615 /local/git/run_qe/test_run/check_input_error_scf_3/sg_15-f_7-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_104-f

0.99999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_93-f_1-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_137-f_0-0/qe.out
0.999999999994 /local/git/run_qe/test_run/check_input_error_scf_3/sg_185-f_0-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_124-f_4-0/qe.out
0.999999999982 /local/git/run_qe/test_run/check_input_error_scf_3/sg_40-f_4-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_1-f_4-0/qe.out
0.999999999987 /local/git/run_qe/test_run/check_input_error_scf_3/sg_110-f_2-0/qe.out
0.999999999983 /local/git/run_qe/test_run/check_input_error_scf_3/sg_108-f_2-0/qe.out
0.999999999986 /local/git/run_qe/test_run/check_input_error_scf_3/sg_21-f_0-0/qe.out
0.999999999987 /local/git/run_qe/test_run/check_input_error_scf_3/sg_20-f_3-0/qe.out
0.999999999996 /local/git/run_qe/test_run/check_input_error_scf_3/sg_151-f_2-0/qe.out
0.999999999983 /local/git/run_qe/test_run/check_input_error_s

1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_162-f_2-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_211-f_0-0/qe.out
0.999999999992 /local/git/run_qe/test_run/check_input_error_scf_3/sg_24-f_0-1/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_91-f_3-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_145-f_1-0/qe.out
0.999999999979 /local/git/run_qe/test_run/check_input_error_scf_3/sg_90-f_1-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_28-f_4-1/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_92-f_1-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_206-f_1-0/qe.out
0.999999999918 /local/git/run_qe/test_run/check_input_error_scf_3/sg_79-f_2-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_138-f_0-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_27-f_3-0/qe.out
0.999999999955 /local/git/run_qe/test_run/check_inpu

0.999999996063 /local/git/run_qe/test_run/check_input_error_scf_3/sg_160-f_1-0/qe.out
0.999999997403 /local/git/run_qe/test_run/check_input_error_scf_3/sg_8-f_3-0/qe.out
0.999999999997 /local/git/run_qe/test_run/check_input_error_scf_3/sg_60-f_4-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_32-f_4-0/qe.out
0.999999999937 /local/git/run_qe/test_run/check_input_error_scf_3/sg_14-f_3-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_140-f_4-0/qe.out
1.0 /local/git/run_qe/test_run/check_input_error_scf_3/sg_153-f_5-0/qe.out
0.999999999994 /local/git/run_qe/test_run/check_input_error_scf_3/sg_127-f_3-0/qe.out
0.999999999998 /local/git/run_qe/test_run/check_input_error_scf_3/sg_193-f_1-0/qe.out
0.999999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_106-f_0-0/qe.out
0.99999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_99-f_0-0/qe.out
0.99999999999 /local/git/run_qe/test_run/check_input_error_scf_3/sg_37-f

In [77]:
lll = []
for it in idd:
    if not np.allclose(gk[it[0],it[1]],1.):
        lll.append(it[2:])
        #print it,gk[it[0],it[1]]

In [78]:
from make_input.raw_info import bravaisLattice2ibrav,SG2BravaisLattice

In [79]:
ssg = range(1,230+1)
sg2ibrav = {}
ibrav2sg = {ibrav:[] for ibrav in bravaisLattice2ibrav.values()}
for sg in ssg:
    bl = SG2BravaisLattice[sg]
    ibrav = bravaisLattice2ibrav[bl]
    ibrav2sg[ibrav].append(sg)
    sg2ibrav[sg] = ibrav

In [91]:
for sg,f in lll: 
    
    crystal = crystals[sg][f]
    symprec = get_symprec(crystal, sg)
    data = spg.get_symmetry_dataset(crystal,symprec=symprec)
    sss = None
    if sg in missClassificationCorrection.keys():
        sss = missClassificationCorrection[sg]
    print sg,f,sg2ibrav[sg],data['wyckoffs'][0],sss

155 2 5 f None
146 4 5 b None
148 5 5 f None
148 6 5 f None
161 0 5 b None
166 1 5 i None
160 0 5 c None
148 3 5 f None
155 0 5 f None
148 1 5 f None
167 3 5 f None
148 7 5 f None
146 2 5 b None
161 4 5 b None
167 7 5 f None
155 4 5 f None
167 2 5 f None
148 0 5 f None
166 4 5 i None
167 6 5 f None
160 3 5 c None
155 3 5 f None
160 4 5 c None
161 2 5 b None
155 1 5 f None
161 3 5 b None
167 5 5 f None
167 9 5 f None
166 3 5 i None
167 8 5 f None
146 3 5 b None
146 0 5 b None
148 2 5 f None
167 0 5 f None
161 5 5 b None
161 1 5 b None
148 4 5 f None
146 1 5 b None
161 6 5 b None
166 2 5 i None
160 2 5 c None
167 1 5 f None
160 1 5 c None
167 4 5 f None


In [366]:
sg = 160
cc1 = crystals[sg][1]
cc2 = crystals[sg][0]
view(cc1)
view(cc2)

In [362]:
for sg,f in lll:
    crystal = crystals[sg][f]
    symprec = get_symprec(crystal, sg)
    data = spg.get_symmetry_dataset(crystal,symprec=symprec)
    print sg,f,sg2ibrav[sg],data['wyckoffs'][0]

155 2 5
146 4 5
148 5 5
148 6 5
161 0 5
166 1 5
76 4 6
32 3 8
31 3 8
148 1 5
135 1 6
197 1 3
137 3 6
41 7 91
208 1 1
27 2 8
23 3 11
61 2 8
63 4 9
74 3 11
129 1 6
43 5 10
34 0 8
173 1 4
153 2 4
139 1 7
229 0 3
178 1 4
16 4 8
63 3 9
76 3 6
10 3 -12
152 5 4
106 2 6
13 2 -12
97 4 7
138 0 6
180 4 4
26 1 8
185 2 4
58 4 8
69 1 10
60 1 8


In [358]:
for k,v in ibrav2sg.iteritems():
    print k,v

1 [195, 198, 200, 201, 205, 207, 208, 212, 213, 215, 218, 221, 222, 223, 224]
2 [196, 202, 203, 209, 210, 216, 219, 225, 226, 227, 228]
3 [197, 199, 204, 206, 211, 214, 217, 220, 229, 230]
4 [143, 144, 145, 147, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159, 162, 163, 164, 165, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194]
5 [146, 148, 155, 160, 161, 166, 167]
6 [75, 76, 77, 78, 81, 83, 84, 85, 86, 89, 90, 91, 92, 93, 94, 95, 96, 99, 100, 101, 102, 103, 104, 105, 106, 111, 112, 113, 114, 115, 116, 117, 118, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138]
7 [79, 80, 82, 87, 88, 97, 98, 107, 108, 109, 110, 119, 120, 121, 122, 139, 140, 141, 142]
8 [16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
9 [20, 21, 35, 36, 37, 63, 64, 65, 66, 67, 68]
10 [22, 42, 43, 69, 70]
11 [23, 24, 44, 45, 46, 71, 72

#  test relax

In [19]:
zatom = 14
calculation_type = '"vc-relax"'
sgs = range(1,30)

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 40*4,40
smearing = 1e-2
etot_conv_thr = 1e-6
forc_conv_thr = 1e-6
nstep = 2
scf_conv_thr = 10
ppPath='"/local/git/run_qe/pseudo/SSSP_acc_PBE/"'

dataPath = '/local/git/run_qe/test_run/check_input_error_relax_2/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [20]:
for (sg,it),dirName in dirNames.items():
    #if sg2ibrav[sg] == -12:
    crystal = crystals[sg][it]
    symprec = get_symprec(crystal, sg)
    #print crystal, (sg,it),dirName
    #print symprec
    try:
        data = spg.get_symmetry_dataset(crystal,symprec=symprec)
        print dirName,'  ',data['wyckoffs'][0]
    except:
        print (sg,it),dirName
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=etot_conv_thr,forc_conv_thr=forc_conv_thr,nstep=nstep,
                    scf_conv_thr=scf_conv_thr,print_forces=True,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

#         print exitstatus

/local/git/run_qe/test_run/check_input_error_relax_2/sg_7-f_3    f
sending the calc
/local/git/run_qe/test_run/check_input_error_relax_2/sg_7-f_3
/local/git/run_qe/test_run/check_input_error_relax_2/sg_12-f_1    j
sending the calc
/local/git/run_qe/test_run/check_input_error_relax_2/sg_12-f_1
/local/git/run_qe/test_run/check_input_error_relax_2/sg_14-f_4    e
sending the calc
/local/git/run_qe/test_run/check_input_error_relax_2/sg_14-f_4
/local/git/run_qe/test_run/check_input_error_relax_2/sg_13-f_4    g
sending the calc
/local/git/run_qe/test_run/check_input_error_relax_2/sg_13-f_4
/local/git/run_qe/test_run/check_input_error_relax_2/sg_19-f_4    a
sending the calc
/local/git/run_qe/test_run/check_input_error_relax_2/sg_19-f_4
/local/git/run_qe/test_run/check_input_error_relax_2/sg_18-f_4    c
sending the calc
/local/git/run_qe/test_run/check_input_error_relax_2/sg_18-f_4
/local/git/run_qe/test_run/check_input_error_relax_2/sg_15-f_1    f
sending the calc
/local/git/run_qe/test_run/ch

KeyboardInterrupt: 

In [12]:
# fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_{}*'.format(sgs[0]))
fns = get_filenames(dataPath,fn_pattern='qe.out',dir_pattern='sg_*')
# fns

In [15]:
fff = []
idd = []

for it,fn in enumerate(fns):
    info = fn2info(fn)
    sg = info['sg']
    f = info['f']
    cc  = read(fn,index=':')
    
    dd = crystals[sg][f]
    #ee = get_standard_frame(dd,sg)
    fff.extend([cc[0],dd])
    idd.append([it*2,(it)*2+1,sg,f])
gk = compare(fff)

for it in idd:
    print it,gk[it[0],it[1]]

Using compiled and threaded version of envKernel function
1 threaded calc
with implicit delta kernel function


HBox(children=(IntProgress(value=0, description=u'SOAP vectors', max=30), HTML(value=u'')))

HBox(children=(IntProgress(value=0, description=u'Env kernels', max=465), HTML(value=u'')))


[0, 1, 5, 1] 0.999999999982
[2, 3, 9, 0] 1.0
[4, 5, 5, 1] 0.999999999982
[6, 7, 15, 6] 0.999999999987
[8, 9, 15, 0] 0.999999999999
[10, 11, 9, 4] 0.999999999994
[12, 13, 8, 0] 0.999999999997
[14, 15, 12, 1] 0.999999999992
[16, 17, 15, 4] 0.999999999991
[18, 19, 8, 2] 1.0
[20, 21, 15, 10] 0.999999999993
[22, 23, 5, 4] 0.999999999999
[24, 25, 9, 0] 1.0
[26, 27, 9, 3] 0.999999999998
[28, 29, 15, 4] 0.999999999991


# Analysis

In [462]:
def isInSP(crystal, spaceGroupIdx, tols=[1e-5, 1e-4, 1e-3, 1e-2]):
    SPgood = False
    detectedSpaceGroup = []

    for tol in tols:
        detectedSpaceGroup.append(crystal2SpaceGroupIdx(crystal, tol))
    print detectedSpaceGroup
    if spaceGroupIdx in detectedSpaceGroup:
        SPgood = True
    elif spaceGroupIdx in missClassificationCorrection.keys():
        if missClassificationCorrection[spaceGroupIdx] in detectedSpaceGroup:
            SPgood = True
    return SPgood
def crystal2SpaceGroupIdx(crystal,symtol):
    '''Identify the space group index (from 1,230) of a given crystal (ase Atoms class).
        Return an int from 1 to 230.'''
    try:
        return int(spg.get_spacegroup(crystal,symprec=symtol).split(' ')[1][1:-1])
    except:

        return -1

In [485]:
for iii in range(11):
    isInSP(crystals[28][iii],28)

[51, 51, 65, 65]
[10, 10, 51, 51]
[10, 10, 51, 191]
[10, 10, 10, 51]
[10, 10, 51, 51]
[25, 51, 47, 47]
[51, 51, 63, 194]
[51, 51, 65, 191]
[10, 51, 51, 65]
[6, 51, 51, 51]
[10, 10, 51, 51]


In [500]:
for it in [16, 17, 18, 19, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 47, 48, 49, 50, 51,
                   52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]:
    print it,SGTable[SGTable['Table No.']==it]['Hall symbol'].values[0]

16 P 2 2
17 P 2c 2
18 P 2 2ab
19 P 2ac 2ab
25 P 2 -2
26 P 2c -2
27 P 2 -2c
28 P 2 -2a
29 P 2c -2ac
30 P 2 -2bc
31 P 2ac -2
32 P 2 -2ab
33 P 2c -2n
34 P 2 -2n
47 -P 2 2
48 P 2 2 -1n
49 -P 2 2c
50 P 2 2 -1ab
51 -P 2a 2a
52 -P 2a 2bc
53 -P 2ac 2
54 -P 2a 2ac
55 -P 2 2ab
56 -P 2ab 2ac
57 -P 2c 2b
58 -P 2 2n
59 P 2 2ab -1ab
60 -P 2n 2ab
61 -P 2ac 2ab
62 -P 2ac 2n


In [72]:
print_sg_table(49,12)

In [552]:
it = 0
aa = missClassificationCorrection.items()
print 'SG left = {}                                                              SG right = {}'.format(aa[it][0],aa[it][1])


sg = aa[it][0]

for ii,cc in enumerate(crystals[sg]):
    symprec = get_symprec(cc,sg)
    data = spg.get_symmetry_dataset(cc,symprec=symprec)
    
    print ii,data['wyckoffs'],data['number']
#     print data['std_positions']
    print cc.get_scaled_positions()
#     print cc.get_cell_lengths_and_angles()
print_sg_table(aa[it][0],aa[it][1])
# print_sg_table(aa[it][0],51)

SG left = 30                                                              SG right = 53
0 ['h', 'h', 'h', 'h'] 53
[[ 0.75001996  0.87499361  0.49      ]
 [ 0.24998004  0.12500639  0.49      ]
 [ 0.75001996  0.62500639  0.99      ]
 [ 0.24998004  0.37499361  0.99      ]]
1 ['h', 'h', 'h', 'h'] 53
[[ 0.2500044   0.37498106  0.556     ]
 [ 0.74999532  0.62501894  0.556     ]
 [ 0.25000468  0.12501894  0.056     ]
 [ 0.7499956   0.87498106  0.056     ]]
2 ['h', 'h', 'h', 'h'] 53
[[ 0.24999719  0.84244628  0.487     ]
 [ 0.75000281  0.15755372  0.487     ]
 [ 0.24999719  0.65755372  0.987     ]
 [ 0.75000281  0.34244628  0.987     ]]
3 ['h', 'h', 'h', 'h'] 53
[[  2.50044220e-01   2.81118353e-06   2.18999991e-01]
 [  7.49955780e-01   9.99997189e-01   2.19000009e-01]
 [  2.50044220e-01   4.99997286e-01   7.18999991e-01]
 [  7.49955780e-01   5.00002714e-01   7.19000009e-01]]
4 ['h', 'h', 'h', 'h'] 53
[[  2.49995007e-01   4.99996722e-01   2.89000002e-01]
 [  7.50004973e-01   5.00003278e-01   2.

In [551]:
sg,it = 29,2
cc = crystals[sg][it]
ee = aspg.crystal(symbols=['Si'],basis=cc.get_scaled_positions()[0],cellpar=cc.get_cell_lengths_and_angles(),spacegroup=sg)
print np.allclose(ee.get_scaled_positions(),cc.get_scaled_positions(),atol=1e-5)
print np.allclose(ee.get_cell(),cc.get_cell(),atol=1e-5)
view(cc)
view(ee)

False
True


In [290]:
wyck2match = {
    3:['m', 'n'],
    4:['e'],
    5:['i'],
    6:['i','j'],
    7:['e','f'],
    8:['g','h'],
    9:['e'],
    25:['z','y'],
    26:['j'],
    27:['q','i','j','k','l'],
    28:['j'],
    29:['d']
    
    
}

In [306]:
cc = crystals[sg][3]
dd = get_standard_frame(cc,sg)
print dd.get_scaled_positions()

[[ 0.75105019  0.5         0.60897755]
 [ 0.24894981  0.5         0.39102245]]


In [307]:
symprec=1e-5
data = spg.get_symmetry_dataset(cc,symprec=symprec)
print data['wyckoffs']
print spg.get_spacegroup(cc,symprec=symprec)
(lattice, positions, numbers) = spg.standardize_cell(
                            cc, to_primitive=False,no_idealize=False,
                            symprec=symprec, angle_tolerance=-1.0)
dd = ase.Atoms(cell=lattice, scaled_positions=positions, numbers=numbers)
print dd.get_scaled_positions()
print cc.get_cell()
print data['std_lattice']

['n', 'n']
P2/m (10)
[[ 0.75105019  0.5         0.60897755]
 [ 0.24894981  0.5         0.39102245]]
[[  2.25872516e+00   3.08179506e-19   2.38582043e-05]
 [  2.41545830e-18   2.23751551e+00   4.52021698e-19]
 [  4.49559691e+00   3.54006381e-18   9.04446916e+00]]
[[ 2.25872516  0.          0.        ]
 [ 0.          2.23751551  0.        ]
 [-0.02175788  0.          9.04442168]]
